In [183]:
evals = list()

In [184]:
import utils.common
import evaluation
import importlib
import numpy as np
import time
from models.ConstraintAutoRec import ConstraintAutoRec 
import models.NeuralLogicRec
import tensorflow as tf
import pandas as pd

importlib.reload(utils.common)
importlib.reload(evaluation)
importlib.reload(models.NeuralLogicRec)
import itertools

ml_small = utils.common.ml_small
ml_big = utils.common.movie_lens


In [185]:
ev = evaluation.Evaluation(ml_small)

In [187]:
nlr = models.NeuralLogicRec.NLR(ml_small['user'], ml_small['dimensions'], epochs=10, embedding_dim=16, batch_size=8)

In [ ]:
nlr.train(utils.common.load_dataset(ml_small), ml_small['train']['records'])
evals.append(ev.evaluate_single_thread(nlr))

Epoch #0 Loss at step 76: 0.2928, time: 7.505. Train accuracy 0.633, Validation accuracy 0.196
Epoch #1 Loss at step 76: 0.2492, time: 4.867. Train accuracy 0.616, Validation accuracy 0.239
Epoch #2 Loss at step 76: 0.2385, time: 4.850. Train accuracy 0.621, Validation accuracy 0.196
Epoch #3 Loss at step 76: 0.2294, time: 4.874. Train accuracy 0.620, Validation accuracy 0.196
Epoch #4 Loss at step 76: 0.2294, time: 4.858. Train accuracy 0.623, Validation accuracy 0.217
Epoch #5 Loss at step 76: 0.2268, time: 4.847. Train accuracy 0.619, Validation accuracy 0.217
Epoch #6 Loss at step 76: 0.2271, time: 4.831. Train accuracy 0.618, Validation accuracy 0.217
Epoch #7 Loss at step 76: 0.2231, time: 4.876. Train accuracy 0.619, Validation accuracy 0.196


In [182]:
pd.DataFrame(evals)

,accuracy,batch_size,diversity@10,diversity@5,epc@10,epc@5,epd@5,epochs,latent_dim,map@1,map@10,map@5,name,precision,recall
0,0.172145,16,0.384387,0.384570,0.984839,0.984496,0.382623,5,128,0.000000,0.000818,0.001231,NeuralLogicRec,0.840777,0.040403
1,0.180093,32,0.385809,0.387973,0.977288,0.971680,0.381561,5,128,0.018018,0.002372,0.004129,NeuralLogicRec,0.868519,0.044821
2,0.178842,32,0.378166,0.378996,0.976318,0.969217,0.378220,5,128,0.013514,0.002508,0.004505,NeuralLogicRec,0.840453,0.052693
3,0.174719,32,0.377060,0.374659,0.977235,0.974359,0.374596,5,128,0.009009,0.002738,0.003288,NeuralLogicRec,0.877130,0.038415
4,0.173082,20,0.372784,0.366913,0.987900,0.988059,0.376050,20,128,0.000000,0.000846,0.001111,NeuralLogicRec,0.881246,0.037240
5,0.187013,8,0.378726,0.378561,0.983886,0.982657,0.380312,20,128,0.009009,0.001678,0.002733,NeuralLogicRec,0.867490,0.056580


In [170]:
# print(np.flip(np.argsort(nlr.predict_single_user(3))))
pred = nlr.model([4])
# inf = models.NeuralLogicRec.map_inference(nlr.model, pred)
print(np.sort(pred['likes']))
print(pred['likes'][:,10:20])
print(pred['rated'][:,10:20])
print(pred['rated'].numpy().mean())
# print(np.sort(inf['likes'].numpy()))
# nlr.model(np.array(1), np.array(1))
# ev.evaluate(nlr)
# pred = nlr.model(tf.convert_to_tensor([1, 2, 3], dtype=tf.int32))

[[0.00142995 0.00326657 0.00341159 ... 0.5646297  0.5646297  0.5646297 ]]
tf.Tensor(
[[0.5646297  0.15429246 0.12063897 0.4047243  0.0911631  0.5646297
  0.5646297  0.30203676 0.43148625 0.11856645]], shape=(1, 10), dtype=float32)
tf.Tensor(
[[0.4458373  0.23893094 0.1800633  0.42173564 0.12459189 0.4524185
  0.45023888 0.28291112 0.4086203  0.14337033]], shape=(1, 10), dtype=float32)
0.10416411


In [85]:
a = [1,2,3]
np.mean(a)

2.0

In [86]:
num_movies = 10
num_user = 3
rated = tf.convert_to_tensor(np.arange(num_movies * num_user).reshape([num_user, num_movies]) % 3 == 0)
seen = tf.convert_to_tensor(np.arange(num_movies * num_user, num_movies * num_user * 2).reshape([num_user, num_movies]))
users = tf.convert_to_tensor(np.arange(num_user))
users = tf.convert_to_tensor([20, 10, 5, 0, 40])
print(rated)
print(seen)

tf.Tensor(
[[ True False False  True False False  True False False  True]
 [False False  True False False  True False False  True False]
 [False  True False False  True False False  True False False]], shape=(3, 10), dtype=bool)
tf.Tensor(
[[30 31 32 33 34 35 36 37 38 39]
 [40 41 42 43 44 45 46 47 48 49]
 [50 51 52 53 54 55 56 57 58 59]], shape=(3, 10), dtype=int64)


In [87]:
user_cross = tf.convert_to_tensor([x for x in itertools.permutations(users.numpy(), 2)])
print(user_cross)
rated_cross = tf.convert_to_tensor([tf.stack([rated[x[0]], rated[x[1]]], axis=0) for x in itertools.permutations(range(len(users.numpy())), 2)])
# print(rated_cross)

tf.Tensor(
[[20 10]
 [20  5]
 [20  0]
 [20 40]
 [10 20]
 [10  5]
 [10  0]
 [10 40]
 [ 5 20]
 [ 5 10]
 [ 5  0]
 [ 5 40]
 [ 0 20]
 [ 0 10]
 [ 0  5]
 [ 0 40]
 [40 20]
 [40 10]
 [40  5]
 [40  0]], shape=(20, 2), dtype=int32)


InvalidArgumentError: slice index 3 of dimension 0 out of bounds. [Op:StridedSlice] name: strided_slice/

In [88]:
len(user_cross)

20

In [89]:
user_emb = tf.expand_dims(tf.nn.embedding_lookup(nlr.model.user_embedding, user_cross), axis=2)
print(user_emb[:, 0])
user_embedd = tf.tile(user_emb, [1, 1, num_movies, 1])
expanded_embedd = tf.reshape(nlr.model.item_embedding, [1, 1, num_movies, 24])
item_embedd = tf.tile(expanded_embedd, [len(user_cross), 2, 1, 1])
print(item_embed.shape)
print(user_embedd.shape)
inp = tf.concat([user_embedd, item_embedd], axis=-1)

tf.Tensor(
[[[ 0.47242978 -0.7581886   0.6176951  -0.5861519   0.3641987
    0.10563103  0.61831975 -0.37575603]]

 [[ 0.47242978 -0.7581886   0.6176951  -0.5861519   0.3641987
    0.10563103  0.61831975 -0.37575603]]

 [[ 0.47242978 -0.7581886   0.6176951  -0.5861519   0.3641987
    0.10563103  0.61831975 -0.37575603]]

 [[ 0.47242978 -0.7581886   0.6176951  -0.5861519   0.3641987
    0.10563103  0.61831975 -0.37575603]]

 [[ 0.32968965  1.3128943   0.4282597  -1.8893449  -1.0829191
   -1.2234583   0.99611586  0.7264877 ]]

 [[ 0.32968965  1.3128943   0.4282597  -1.8893449  -1.0829191
   -1.2234583   0.99611586  0.7264877 ]]

 [[ 0.32968965  1.3128943   0.4282597  -1.8893449  -1.0829191
   -1.2234583   0.99611586  0.7264877 ]]

 [[ 0.32968965  1.3128943   0.4282597  -1.8893449  -1.0829191
   -1.2234583   0.99611586  0.7264877 ]]

 [[ 0.6136244   1.5436143   2.036422    1.5196469   0.08901728
   -0.90769035 -1.6418009  -0.3131709 ]]

 [[ 0.6136244   1.5436143   2.036422    1.5196469   

InvalidArgumentError: Input to reshape is a tensor with 83048 values, but the requested shape has 240 [Op:Reshape]

In [90]:
result = tf.squeeze(nlr.model.likes_estimator(inp))

NameError: name 'inp' is not defined

In [84]:
num_movies = 5
users = tf.convert_to_tensor([1, 10, 100])
embed_user = tf.expand_dims(users, axis=1)
users_1 = tf.tile(tf.expand_dims(embed_user, axis=1), [1, len(users), 1])
users_2 = tf.tile(tf.expand_dims(embed_user, axis=0), [len(users),1, 1])
# print(embed_user)
# embed_user = tf.tile(embed_user, [1, num_movies, 1])
# expanded_embedd = tf.expand_dims(nlr.model.item_embedding[0:num_movies], axis=0)
# embed_item = tf.tile(expanded_embedd, [len(users), 1, 1])
sim_input = tf.reduce_sum(tf.concat([users_1, users_2], axis=-1), axis=-1)
print(sim_input)

tf.Tensor(
[[  2  11 101]
 [ 11  20 110]
 [101 110 200]], shape=(3, 3), dtype=int32)


In [99]:
multi_ratings = tf.tile(tf.expand_dims(rated, axis=0), [3, 1, 1])

In [13]:
a = tf.convert_to_tensor(10)
# b = tf.expand_dims(tf.convert_to_tensor(20), axis=0)
# c = tf.convert_to_tensor([1,2,3,4])
len(a.shape) == 0

True

In [117]:
a = np.array([1,5,3,0], dtype=float)
b = np.array([0,2,3,0], dtype=float)
mask = np.where(b)
mask

(array([1, 2]),)

In [118]:
a[mask] / b[mask]

array([2.5, 1. ])